# IMPORTS

In [1]:
import json
import os
import time
from AxonDeepSeg.config_tools import *

## Setting hyperparameters and creating the file _config_

Quick explanations

In [2]:
# Description du fichier config :
# network_learning_rate : float : No idea, but certainly linked to the back propagation ? Default : 0.0005.
# network_n_classes : int : number of labels in the output. Default : 2.
# dropout : float : between 0 and 1 : percentage of neurons we want to keep. Default : 0.75.
# network_depth : int : number of layers WARNING : factualy, there will be 2*network_depth layers. Default : 6.
# network_convolution_per_layer : list of int, length = network_depth : number of convolution per layer. Default : [1 for i in range(network_depth)].
# network_size_of_convolutions_per_layer : list of lists of int [number of layers[number_of_convolutions_per_layer]] : Describe the size of each convolution filter.
# Default : [[3 for k in range(network_convolution_per_layer[i])] for i in range(network_depth)].

# network_features_per_convolution : list of lists of int [number of layers[number_of_convolutions_per_layer[2]] : Numer of different filters that are going to be used.
# Default : [[[64,64] for k in range(network_convolution_per_layer[i])] for i in range(network_depth)]. WARNING ! network_features_per_convolution[k][1] = network_features_per_convolution[k+1][0].

# network_trainingset : string : String describing the dataset used for the training.

Name of the .json file where the network configuration will be stored

In [3]:
filename = '/config_network.json'

## Setting the main parameters of the network

Print default configuration:

In [4]:
default_configuration()

{'network_additional_parameters': {'batch_norm_decay_decay_activate': True,
  'batch_norm_decay_decay_period': 4800,
  'batch_norm_decay_ending_decay': 0.9,
  'learning_rate_decay_activate': True,
  'learning_rate_decay_period': 120,
  'learning_rate_decay_rate': 0.99},
 'network_batch_norm': True,
 'network_batch_norm_decay': 0.7,
 'network_batch_size': 8,
 'network_convolution_per_layer': [2, 2, 2, 2],
 'network_data_augmentation': {'transformations': {'0_shifting': True,
   '1_rescaling': True,
   '2_random_rotation': False,
   '3_elastic': True,
   '4_flipping': True,
   '5_noise_addition': False},
  'type': 'all'},
 'network_depth': 4,
 'network_downsampling': 'convolution',
 'network_dropout': 0.75,
 'network_features_per_convolution': [[[1, 16], [16, 16]],
  [[16, 32], [32, 32]],
  [[32, 64], [64, 64]],
  [[64, 128], [128, 128]]],
 'network_learning_rate': 0.001,
 'network_n_classes': 3,
 'network_size_of_convolutions_per_layer': [[5, 5, 5],
  [3, 3, 3],
  [3, 3, 3],
  [3, 3, 3]

We then set the main parameters and the architecture of the network that we want to change, compared to the default configuration above.

In [5]:
L_struct = [{'structure':[[5,5], [3,3], [3,3], [3,3]], 'features_augmentation':'x2', 'network_first_num_features':16}]

dict_params = {'network_batch_norm':False,
               'network_weighted_cost':True,
               'network_weighted_cost_parameters': [{'boundaries_activate':False,
                                                    'balanced_activate':True}]
              }

config_list = grid_config(L_struct, dict_params)

In [6]:
config_list

{'cv_3c_d4_c2_k3__0-8260': {'network_additional_parameters': {'batch_norm_decay_decay_activate': True,
   'batch_norm_decay_decay_period': 4800,
   'batch_norm_decay_ending_decay': 0.9,
   'learning_rate_decay_activate': True,
   'learning_rate_decay_period': 120,
   'learning_rate_decay_rate': 0.99},
  'network_batch_norm': False,
  'network_batch_norm_decay': 0.7,
  'network_batch_size': 8,
  'network_convolution_per_layer': [2, 2, 2, 2],
  'network_data_augmentation': {'transformations': {'0_shifting': True,
    '1_rescaling': True,
    '2_random_rotation': False,
    '3_elastic': True,
    '4_flipping': True,
    '5_noise_addition': False},
   'type': 'all'},
  'network_depth': 4,
  'network_downsampling': 'convolution',
  'network_dropout': 0.75,
  'network_features_per_convolution': [[[1, 16], [16, 16]],
   [[16, 32], [32, 32]],
   [[32, 64], [64, 64]],
   [[64, 128], [128, 128]]],
  'network_learning_rate': 0.001,
  'network_n_classes': 3,
  'network_size_of_convolutions_per_lay

## Actually training the model

By default, the name is the current date. You can uncomment the second line dir_name if you want to give a specific name to the folder, but beware: it will erase the contents of the folder if it already exists !

The model data will be stored in a folder with this name, as well as the summaries that are going to be readable on tensorboard.

In [7]:
#dir_name = time.strftime("%Y-%m-%d") + '_' + time.strftime("%H-%M-%S") 
#dir_name = 'cv_2c_nw_d5_2_b8_p10-10'
#dir_name += '_' + str(int(time.time()))[-4:]

for dir_name, config in config_list.iteritems():
    pass
path_model = os.path.join('../models/', dir_name)

if not os.path.exists(path_model):
    os.makedirs(path_model)
    
path_model_init = None 


# Specify here the path to the initial model if needed.

if os.path.exists(path_model+filename): # if there is already a configfile
    with open(path_model+filename, 'r') as fd:
        config_network = json.loads(fd.read())
else: # There is no config file for the moment
    with open(path_model+filename, 'w') as f:
        json.dump(config, f, indent=2)
    with open(path_model+filename, 'r') as fd:
        config_network = json.loads(fd.read())
        
        
# Ensure that the config file has all the necessary parameters

config_network = update_config(default_configuration(), config_network)
path_trainingset = os.path.join('../data/', config_network["network_trainingset"], 'training/')

print 'Chosen parameters :\n'

for param_name, param_value in config_network.iteritems():
    print param_name, ' - ', param_value

print '\n-----------------------------------------------------------------\n'
print 'Training model ' + str(dir_name) + '...\n'

################################################### Training phase ###################################################

# Note : possible values for gpu: None, '/gpu:0'. If used withing CUDA_VISIBLE_DEVICES=?, then it will use gpu number ? by default
from AxonDeepSeg.train_network import train_model
train_model(path_trainingset, path_model, config_network, path_model_init=path_model_init,gpu='/gpu:0')

Chosen parameters :

network_additional_parameters  -  {'learning_rate_decay_period': 120, 'batch_norm_decay_decay_period': 4800, 'batch_norm_decay_decay_activate': True, 'learning_rate_decay_activate': True, 'learning_rate_decay_rate': 0.99, 'batch_norm_decay_ending_decay': 0.9}
network_depth  -  4
network_weighted_cost  -  True
network_weighted_cost_parameters  -  {'balanced_activate': True, 'balanced_weights': [1.1, 1, 1.3], 'boundaries_sigma': 2, 'boundaries_activate': False}
network_batch_norm  -  False
network_batch_norm_decay  -  0.7
network_learning_rate  -  0.001
network_downsampling  -  convolution
network_batch_size  -  8
network_data_augmentation  -  {'type': u'all', 'transformations': {'2_random_rotation': False, '5_noise_addition': False, '3_elastic': True, '0_shifting': True, '4_flipping': True, '1_rescaling': True}}
network_n_classes  -  3
network_thresholds  -  [0, 0.2, 0.8]
network_trainingset  -  SEM_3classes_reduced
network_size_of_convolutions_per_layer  -  [[5, 5]

/home/GRAMES.POLYMTL.CA/mawab/ads/tf11-ads/local/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "




----Scores on validation:---
----
 Last epoch: 1
 Accuracy: 0.556395934976;
 Loss: 0.855224954045;


----Scores on validation:---
----
 Last epoch: 2
 Accuracy: 0.585751165514;
 Loss: 0.822047101415;


----Scores on validation:---
----
 Last epoch: 3
 Accuracy: 0.590440291425;
 Loss: 0.812105956285;


----Scores on validation:---
----
 Last epoch: 4
 Accuracy: 0.622373653495;
 Loss: 0.814032124436;


----Scores on validation:---
----
 Last epoch: 5
 Accuracy: 0.663742210554;
 Loss: 0.757660951303;


----Scores on validation:---
----
 Last epoch: 6
 Accuracy: 0.639690440634;
 Loss: 0.792233324569;
Model saved in file: ../models/cv_3c_d4_c2_k3__0-8260/model.ckpt


----Scores on validation:---
----
 Last epoch: 7
 Accuracy: 0.681493481864;
 Loss: 0.722957144613;


----Scores on validation:---
----
 Last epoch: 8
 Accuracy: 0.697772023471;
 Loss: 0.70133631644;


----Scores on validation:---
----
 Last epoch: 9
 Accuracy: 0.701447396175;
 Loss: 0.687246255253;


----Scores on validation:



----Scores on validation:---
----
 Last epoch: 75
 Accuracy: 0.814919793088;
 Loss: 0.432124175455;
Model saved in file: ../models/cv_3c_d4_c2_k3__0-8260/model.ckpt


----Scores on validation:---
----
 Last epoch: 76
 Accuracy: 0.804273141467;
 Loss: 0.452387505251;


----Scores on validation:---
----
 Last epoch: 77
 Accuracy: 0.815719827362;
 Loss: 0.427145110524;


----Scores on validation:---
----
 Last epoch: 78
 Accuracy: 0.809003946574;
 Loss: 0.444397733263;


----Scores on validation:---
----
 Last epoch: 79
 Accuracy: 0.815875138925;
 Loss: 0.42535979981;


----Scores on validation:---
----
 Last epoch: 80
 Accuracy: 0.817987449791;
 Loss: 0.427686299967;


----Scores on validation:---
----
 Last epoch: 81
 Accuracy: 0.822238660377;
 Loss: 0.419464717741;
Model saved in file: ../models/cv_3c_d4_c2_k3__0-8260/model.ckpt


----Scores on validation:---
----
 Last epoch: 82
 Accuracy: 0.808707387551;
 Loss: 0.444129001835;


----Scores on validation:---
----
 Last epoch: 83
 Ac



----Scores on validation:---
----
 Last epoch: 148
 Accuracy: 0.826573581799;
 Loss: 0.406080892553;


----Scores on validation:---
----
 Last epoch: 149
 Accuracy: 0.822159761968;
 Loss: 0.415150992248;


----Scores on validation:---
----
 Last epoch: 150
 Accuracy: 0.825768836167;
 Loss: 0.408933288377;
Model saved in file: ../models/cv_3c_d4_c2_k3__0-8260/model.ckpt


----Scores on validation:---
----
 Last epoch: 151
 Accuracy: 0.827404136243;
 Loss: 0.409257498772;


----Scores on validation:---
----
 Last epoch: 152
 Accuracy: 0.815989870092;
 Loss: 0.426026676012;


----Scores on validation:---
----
 Last epoch: 153
 Accuracy: 0.818614436233;
 Loss: 0.426382042792;


----Scores on validation:---
----
 Last epoch: 154
 Accuracy: 0.820015492647;
 Loss: 0.424855541924;


----Scores on validation:---
----
 Last epoch: 155
 Accuracy: 0.820431527884;
 Loss: 0.417564490567;


----Scores on validation:---
----
 Last epoch: 156
 Accuracy: 0.821309276249;
 Loss: 0.416741718417;
Model sa

...



----Scores on validation:---
----
 Last epoch: 440
 Accuracy: 0.838939801506;
 Loss: 0.375676275595;


----Scores on validation:---
----
 Last epoch: 441
 Accuracy: 0.840372124444;
 Loss: 0.378351836101;


----Scores on validation:---
----
 Last epoch: 442
 Accuracy: 0.841553599938;
 Loss: 0.37302571276;


----Scores on validation:---
----
 Last epoch: 443
 Accuracy: 0.839155334493;
 Loss: 0.378652602434;
Model saved in file: ../models/cv_3c_d4_c2_k3__0-8260/model.ckpt


----Scores on validation:---
----
 Last epoch: 444
 Accuracy: 0.839964081412;
 Loss: 0.376729375642;


----Scores on validation:---
----
 Last epoch: 445
 Accuracy: 0.828730292942;
 Loss: 0.39815974106;


----Scores on validation:---
----
 Last epoch: 446
 Accuracy: 0.840667286645;
 Loss: 0.378936785719;


----Scores on validation:---
----
 Last epoch: 447
 Accuracy: 0.83669014599;
 Loss: 0.387645638507;


----Scores on validation:---
----
 Last epoch: 448
 Accuracy: 0.826248433279;
 Loss: 0.397821808639;


----Score

Model saved in file: ../models/cv_3c_d4_c2_k3__0-8260/model.ckpt


----Scores on validation:---
----
 Last epoch: 513
 Accuracy: 0.839769726214;
 Loss: 0.376091524311;


----Scores on validation:---
----
 Last epoch: 514
 Accuracy: 0.842736635519;
 Loss: 0.372956833114;


----Scores on validation:---
----
 Last epoch: 515
 Accuracy: 0.840635478497;
 Loss: 0.373069910899;


----Scores on validation:---
----
 Last epoch: 516
 Accuracy: 0.832823014778;
 Loss: 0.39524074741;


----Scores on validation:---
----
 Last epoch: 517
 Accuracy: 0.844847583252;
 Loss: 0.369362765032;


----Scores on validation:---
----
 Last epoch: 518
 Accuracy: 0.841115174086;
 Loss: 0.373616729094;
Model saved in file: ../models/cv_3c_d4_c2_k3__0-8260/model.ckpt


----Scores on validation:---
----
 Last epoch: 519
 Accuracy: 0.844800459302;
 Loss: 0.367092924274;


----Scores on validation:---
----
 Last epoch: 520
 Accuracy: 0.839524199133;
 Loss: 0.380559707465;


----Scores on validation:---
----
 Last epoch



----Scores on validation:---
----
 Last epoch: 586
 Accuracy: 0.844781328803;
 Loss: 0.369022237218;


----Scores on validation:---
----
 Last epoch: 587
 Accuracy: 0.842714664729;
 Loss: 0.372502034125;
Model saved in file: ../models/cv_3c_d4_c2_k3__0-8260/model.ckpt


----Scores on validation:---
----
 Last epoch: 588
 Accuracy: 0.846204910589;
 Loss: 0.367408879425;


----Scores on validation:---
----
 Last epoch: 589
 Accuracy: 0.843578431917;
 Loss: 0.37185666613;


----Scores on validation:---
----
 Last epoch: 590
 Accuracy: 0.842476458653;
 Loss: 0.369761805172;


----Scores on validation:---
----
 Last epoch: 591
 Accuracy: 0.844396417556;
 Loss: 0.372667470704;


----Scores on validation:---
----
 Last epoch: 592
 Accuracy: 0.843614254309;
 Loss: 0.371132289586;


----Scores on validation:---
----
 Last epoch: 593
 Accuracy: 0.841028291246;
 Loss: 0.382545147253;
Model saved in file: ../models/cv_3c_d4_c2_k3__0-8260/model.ckpt


----Scores on validation:---
----
 Last epoch



----Scores on validation:---
----
 Last epoch: 659
 Accuracy: 0.845532298088;
 Loss: 0.36969920345;


----Scores on validation:---
----
 Last epoch: 660
 Accuracy: 0.844915926456;
 Loss: 0.368733351645;


----Scores on validation:---
----
 Last epoch: 661
 Accuracy: 0.845944902171;
 Loss: 0.369543209024;


----Scores on validation:---
----
 Last epoch: 662
 Accuracy: 0.846181773621;
 Loss: 0.36525308438;
Model saved in file: ../models/cv_3c_d4_c2_k3__0-8260/model.ckpt


----Scores on validation:---
----
 Last epoch: 663
 Accuracy: 0.833290719468;
 Loss: 0.394571262857;


----Scores on validation:---
----
 Last epoch: 664
 Accuracy: 0.840261296086;
 Loss: 0.380105146895;


----Scores on validation:---
----
 Last epoch: 665
 Accuracy: 0.844657117906;
 Loss: 0.366895455381;


----Scores on validation:---
----
 Last epoch: 666
 Accuracy: 0.842542142972;
 Loss: 0.374125107475;


----Scores on validation:---
----
 Last epoch: 667
 Accuracy: 0.845393582531;
 Loss: 0.365444705538;


----Scor

KeyboardInterrupt: 